In [1]:
import pandas as pd
data_x = pd.read_csv('final_x.csv')  
data_y = pd.read_csv('before_standard_y.csv')  
# data_info = pd.read_csv('x_names_final.csv')

In [2]:
import numpy as np
columns = list(data_x.columns)
data_x_numpy = data_x.to_numpy()
data_y_numpy = data_y.to_numpy()

In [3]:
delete_index_x = []
delete_index_y = []

for i in range(len(columns)):
    if 'Unnamed' in data_x.columns[i]:
        delete_index_x.append(i)
    elif data_x.columns[i] == 'id':
        delete_index_x.append(i)

delete_index_x.reverse()    
for j in delete_index_x:
    data_x_numpy = np.delete(data_x_numpy, j, 1)
    del columns[j]
    
for m in range(len(list(data_y.columns))):
    if 'Unnamed' in data_y.columns[m]:
        delete_index_y.append(m)

delete_index_y.reverse()    
for n in delete_index_y:
    data_y_numpy = np.delete(data_y_numpy, n, 1)

In [4]:
data_x_numpy.shape, data_y_numpy.shape, len(columns)

((131624, 3967), (131624, 2), 3967)

## Feature Selection
使用Validation Threshold，threshold選0.9

In [5]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.99 * (1 - .99)))
X = sel.fit_transform(data_x_numpy)

In [6]:
X.shape

(131624, 294)

In [7]:
selected_column = []
for i in range(len(sel.get_support())):
    if sel.get_support()[i] == True:
        selected_column.append(columns[i])

In [8]:
selected_column

['appearDate',
 'enviornmentPic',
 'driverLicense',
 'other',
 'welfare',
 'jobDescription',
 'businessTrip',
 'workPeriod',
 'edu_不拘',
 'edu_大學',
 'edu_大學、碩士',
 'edu_大學以上',
 'edu_專科、大學',
 'edu_專科、大學、碩士',
 'edu_專科以上',
 'edu_高中',
 'edu_高中、專科',
 'edu_高中、專科、大學',
 'edu_高中、專科、大學、碩士',
 'edu_高中以上',
 'edu_高中以下、高中、專科、大學',
 'major_一般商業學類',
 'major_商業及管理學科類',
 'major_土木工程相關',
 'major_工程學科類',
 'major_會計學相關',
 'major_機械工程相關',
 'major_機械維護相關',
 'major_護理助產相關',
 'major_資訊工程相關',
 'major_資訊管理相關',
 'major_電機電子工程相關',
 'major_電機電子維護相關',
 'language_中文',
 'language_日文',
 'language_英文',
 'specialty_Adobe Photoshop',
 'specialty_AutoCAD',
 'specialty_Excel',
 'specialty_Illustrator',
 'specialty_Outlook',
 'specialty_PowerPoint',
 'specialty_Word',
 'specialty_中文打字20~50',
 'specialty_中文打字50~75',
 'specialty_英文打字20~50',
 'skill_報表彙整與管理',
 'skill_客戶情報蒐集',
 'skill_客戶資料更新維護',
 'skill_客訴案件處理',
 'skill_文件或資料輸入建檔處理',
 'skill_文件收發與檔案管理',
 'skill_文件檔案資料處理、轉換及整合工作',
 'skill_文書處理╱排版能力',
 'skill_業務或通路開發',
 'skill_業績目標分配與

## Split Train, Valid, Stack and Test

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data_y_numpy, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [10]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_valid shape:', X_valid.shape)
print('y_valid shape:', y_valid.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (84239, 294)
y_train shape: (84239, 2)
X_valid shape: (21060, 294)
y_valid shape: (21060, 2)
X_test shape: (26325, 294)
y_test shape: (26325, 2)


## Linear Regression

In [11]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
reg = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)

In [12]:
ypred = reg.predict(X_test)

reg_rmse = []
reg_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
reg_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

reg_mae = []
reg_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
reg_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

print('test rmse:', reg_rmse)
print('test mae: ', reg_mae)

test rmse: [5612.927925179542, 18760.63329381766]
test mae:  [3287.9468423551757, 9938.916448243115]


## Lasso
tune alpha(regularization strength)
有Convergence Warning，將max iter設為1000，並關閉warning

In [13]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')

lasso_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
lasso_rmse = np.zeros((len(lasso_alpha), 2))
lasso_mae = np.zeros((len(lasso_alpha), 2))

for i in range(len(lasso_alpha)):
    lasso = MultiOutputRegressor(linear_model.Lasso(alpha=lasso_alpha[i], max_iter=1000)).fit(X_train, y_train)
    ypred = lasso.predict(X_valid)
    lasso_rmse[i, 0] = np.mean(np.square(y_valid[:, 0] - ypred[:, 0])) ** 0.5
    lasso_rmse[i, 1] = np.mean(np.square(y_valid[:, 1] - ypred[:, 1])) ** 0.5
    lasso_mae[i, 0] = np.mean(np.abs(y_valid[:, 0] - ypred[:, 0]))
    lasso_mae[i, 1] = np.mean(np.abs(y_valid[:, 1] - ypred[:, 1]))

In [14]:
lasso_rmse, lasso_mae

(array([[ 5400.64311951, 18541.94800812],
        [ 5400.63912604, 18541.94366784],
        [ 5400.60112427, 18541.90040738],
        [ 5400.29775448, 18541.48191113],
        [ 5401.26801742, 18538.91221273],
        [ 5449.65969504, 18557.243134  ],
        [ 5822.55311777, 19049.96063197]]),
 array([[ 3256.18618833,  9962.98742356],
        [ 3256.1762966 ,  9962.97375067],
        [ 3256.07789763,  9962.83703288],
        [ 3255.16625337,  9961.50496957],
        [ 3250.44970444,  9949.59011192],
        [ 3269.54550098,  9876.8116389 ],
        [ 3635.07728328, 10044.14729425]]))

In [15]:
lasso_index = np.argmin(np.sum(lasso_rmse, axis=1))
best_lasso_alpha = lasso_alpha[lasso_index]

In [16]:
best_lasso_alpha

1

選rmse加總最小者

In [17]:
best_lasso = MultiOutputRegressor(linear_model.Lasso(alpha=best_lasso_alpha)).fit(X_train, y_train)
ypred = best_lasso.predict(X_test)

best_lasso_rmse = []
best_lasso_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_lasso_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

best_lasso_mae = []
best_lasso_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_lasso_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [18]:
best_lasso_rmse, best_lasso_mae

([5613.314146686907, 18756.210906227832],
 [3282.4939082234373, 9923.116043938091])

## Ridge
tune alpha(regularization strength)

In [19]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model

ridge_rmse = np.zeros((len(lasso_alpha), 2))
ridge_mae = np.zeros((len(lasso_alpha), 2))
ridge_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

for i in range(len(ridge_alpha)):
    ridge = linear_model.Ridge(alpha=i).fit(X_train, y_train)
    ypred = ridge.predict(X_valid)
    ridge_rmse[i, 0] = np.mean(np.square(y_valid[:, 0] - ypred[:, 0])) ** 0.5
    ridge_rmse[i, 1] = np.mean(np.square(y_valid[:, 1] - ypred[:, 1])) ** 0.5
    ridge_mae[i, 0] = np.mean(np.abs(y_valid[:, 0] - ypred[:, 0]))
    ridge_mae[i, 1] = np.mean(np.abs(y_valid[:, 1] - ypred[:, 1]))

In [20]:
ridge_rmse, ridge_mae

(array([[ 5399.93564937, 18542.2096407 ],
        [ 5400.0132462 , 18541.4736524 ],
        [ 5400.1216273 , 18541.47767159],
        [ 5400.19043599, 18541.5050127 ],
        [ 5400.23795265, 18541.5438506 ],
        [ 5400.27226079, 18541.59051404],
        [ 5400.29769427, 18541.64347373]]),
 array([[3255.7898623 , 9964.6331434 ],
        [3255.61617954, 9962.4787278 ],
        [3255.56327626, 9961.77929474],
        [3255.50074385, 9961.20581676],
        [3255.4307479 , 9960.6697647 ],
        [3255.35639937, 9960.15925162],
        [3255.27982884, 9959.65079322]]))

In [21]:
ridge_index = np.argmin(np.sum(ridge_rmse, axis=1))
best_ridge_alpha = ridge_alpha[ridge_index]

In [22]:
best_ridge_alpha

0.001

In [23]:
best_ridge = MultiOutputRegressor(linear_model.Ridge(alpha=best_ridge_alpha)).fit(X_train, y_train)
ypred = best_ridge.predict(X_test)

best_ridge_rmse = []
best_ridge_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_ridge_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

best_ridge_mae = []
best_ridge_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_ridge_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [24]:
best_ridge_rmse, best_ridge_mae

([5612.992453328786, 18760.698725910464],
 [3287.9259146844865, 9938.866557944688])

## Random Forest
tune max depth, n_estimators

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {'estimator__n_estimators':np.arange(5,1000,10),
              'estimator__max_depth': [5, 10, 20]}

model = RSCV(MultiOutputRegressor(RandomForestRegressor()), param_grid, n_iter = 15).fit(X_train, y_train)
best_random_forest = model.best_estimator_

In [ ]:
ypred = best_random_forest.predict(X_test)
best_forest_rmse = []
best_forest_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_forest_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)
best_forest_mae = []
best_forest_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_forest_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [ ]:
best_forest_rmse, best_forest_mae

In [ ]:
model.best_estimator_